In [ ]:
import pandas as pd
from pandas.api.types import CategoricalDtype as Category
import matplotlib.pyplot as plt
from glob import glob
import seaborn as sns
import numpy as np
from iterextras import par_for
import subprocess as sp
import os
import itertools
from tqdm.auto import tqdm
import rs_utils

sns.set()

In [ ]:
def cond(df, m=None, c=None, p=None):
    if m is not None:
        df = df[df.mutability_mode == m]
    if c is not None:
        df = df[df.context_mode == c]
    if p is not None:
        df = df[df.pointer_mode == p]
    return df

In [ ]:
PATHS = glob('../data/*.json')
CRATES = [os.path.splitext(os.path.basename(path))[0] for path in PATHS]
CATEGORIES = {
    'mutability_mode': Category(["DistinguishMut", "IgnoreMut"]),
    'context_mode': Category(["Recurse", "SigOnly"]),
    'pointer_mode': Category(["Conservative", "Precise"]),
    'crate': Category(CRATES)
}

def parse_data(path):
    print(path)
    if os.stat(path).st_size == 0:
        return []
    
    crate = os.path.splitext(os.path.basename(path))[0]
    df = pd.DataFrame(rs_utils.parse_data(path))
    df['crate'] = crate
    for k, dt in CATEGORIES.items():
        df[k] = df[k].astype(dt)
    return df


all_dfs = par_for(parse_data, PATHS)
df = pd.concat(all_dfs).reset_index()
del all_dfs

df = df[(df.num_relevant_instructions != 0)]
df['ins_rel_nonzero'] = df.instructions_relative > 0

In [ ]:
len(df.function_path.unique())

In [ ]:
len(df)/8

In [ ]:
df['ins_rel_frac_log'] = np.log(df.instructions_relative_frac)

# Dataset summary

In [ ]:
#df2 = df.groupby(['crate', 'function_path']).apply(compute_loc)#.groupby('crate').sum()
g = df2.reset_index().groupby('crate')
out = g.sum()
out['size'] = g.size()
out

## Execution time

In [ ]:
df2 = cond(df, m='DistinguishMut', c='SigOnly', p='Precise')
print(df2.duration.max(), df2.duration.median() * 100000)


In [ ]:
df2 = cond(df, m='IgnoreMut', c='Recurse', p='Conservative')
print(df2.duration.max(), df2.duration.median() * 100000)

## Number of instructions

In [ ]:
ax = sns.histplot(df.num_instructions)
# ax.set_xscale('log')

In [ ]:
df_big = df[df.num_instructions >= 1000]
len(df_big) / len(df)

# Slice size analysis

In [ ]:
def plot_ins(data, xscale='linear', yscale='linear', field='instructions_relative_frac', nz=True, ax=None, **kwargs):
    if xscale == 'log':
        df_plot = data[data.ins_rel_nonzero]
        bins = np.logspace(np.log10(0.00001), np.log10(df_plot[field].max()))
    elif xscale == 'symlog':
        df_plot = data
        min_n = df_plot[df_plot[field] > 0][field].min()/10
        bins = np.concatenate(([0], np.logspace(np.log10(min_n), np.log10(df_plot[field].max()))))
    else:
        df_plot = data
        n = df_plot.instructions_relative_frac.max()
        bins = np.arange(0, n, n / 50)
        
    ax = sns.histplot(df_plot[field], bins=bins, ax=ax) #element='step')
    ax.set_yscale(yscale)
    if xscale == 'symlog':
        ax.set_xscale(xscale, linthresh=min_n, linscale=0.2)
    else:
        ax.set_xscale(xscale)
        
    ax.set_xlabel('\% diff in slice size' + (', log scale' if 'log' in xscale else ''))
    ax.set_ylabel('Count' + (', log scale' if yscale == 'log' else ''))
    
    if xscale == 'log' and nz:
        n = data[data.ins_rel_nonzero][field].min() / 10
        ax.bar(n, len(data[~data.ins_rel_nonzero]), color='r', width=[n])
    elif xscale == 'symlog':
        ax.set_xlim(0)
        xticks = ax.get_xticks()
        ax.set_xticks(np.concatenate((xticks[0:1], xticks[3:])))
        
plot_ins_nz = lambda *args, **kwargs: plot_ins(*args, xscale='log', yscale='linear', nz=False, **kwargs)
plot_ins_abs = lambda *args, **kwargs: plot_ins(*args, field='instructions_relative', **kwargs)

# plot_ins(df, 'symlog', 'log')

In [ ]:
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Linux Libertine O']})
rc('text', **{'usetex': True, 'latex.preamble': r'\usepackage{libertine}\usepackage[libertine]{newtxmath}'})

df2 = cond(df, c='SigOnly', m='DistinguishMut', p='Precise')
fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=(9, 2.5))
plot_ins(df2, ax=ax1)
ax1.set_title('\\textsc{x-linear / y-linear}')
ax1.set_yticklabels(['0', '$2 \cdot 10^6$', '$4 \cdot 10^6$'])
# plot_ins(cond(df, c='SigOnly', m='DistinguishMut', p='Precise'), yscale='log', ax=ax2)
# ax2.set_title('\\textsc{x-linear / y-log}')
plot_ins(df2, xscale='symlog', yscale='log', ax=ax2)
ax2.set_title('\\textsc{x-log / y-log}')
plot_ins(df2, xscale='log', nz=False, ax=ax3)
ax3.set_title('\\textsc{x-log / y-linear (no zeros)}')
ax3.set_xticks([10e-6, 10e-4, 10e-2, 10e-0, 10e2])
# fig.suptitle('Distribution of slice \%diff for SigOnly vs. Recurse')
fig.tight_layout()
fig.savefig('eval-recurse-dist.pdf')

In [ ]:
fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=(9, 2.5), sharey=True)

plot_ins(cond(df, c='SigOnly', m='DistinguishMut', p='Precise'), xscale='log', nz=False, ax=ax1)
ax1.set_title('\\textsc{Baseline} vs. \\textsc{Whole-program}')
plot_ins(cond(df, c='SigOnly', m='IgnoreMut', p='Precise'), field='instructions_relative_base_frac', 
         xscale='log', nz=False, ax=ax2)
ax2.set_title('\\textsc{Mut-blind} vs. \\textsc{Baseline}')
plot_ins(cond(df, c='SigOnly', m='DistinguishMut', p='Conservative'), field='instructions_relative_base_frac', 
         xscale='log', nz=False, ax=ax3)
ax3.set_title('\\textsc{Pointer-blind} vs. \\textsc{Baseline}')

ax1.set_xlabel('')
ax3.set_xlabel('')

fig.tight_layout()

fig.savefig('eval-all-dist.pdf')

In [ ]:
df2 = cond(df, c='SigOnly', m='IgnoreMut', p='Precise')
g = df2[df2.instructions_relative_base_frac > 0].groupby('crate').size().rename('nz')
g2 = pd.concat([
    g, 
    df2.groupby('crate').size().rename('total'), 
    df2.groupby('crate').num_instructions.median().rename('avg_ins')], axis=1)
g2['frac'] = g2.nz / g2.total
g2.sort_values('frac')

In [ ]:
g = sns.FacetGrid(data=cond(df, c='SigOnly', m='IgnoreMut', p='Precise'), 
                  col='crate', col_wrap=5, sharex=False, height=2.5, aspect=0.9)
g.map_dataframe(plot_ins, xscale='log', nz=False, field='instructions_relative_base_frac')
g.set_titles('\\textsc{{{col_name}}}')
g.fig.supylabel('Count')
g.fig.supxlabel('\% diff in slice size, log scale')
g.fig.tight_layout()
g.fig.savefig('eval-crates.pdf')

In [ ]:
from scipy.stats.mstats import gmean

df2 = cond(df, c='SigOnly', m='IgnoreMut', p='Precise')
nz = df2[df2.instructions_relative_base_frac > 0]
print(len(nz) / len(df2), gmean(nz.instructions_relative_base_frac))

In [ ]:
df2 = cond(df, c='SigOnly', m='DistinguishMut', p='Conservative')
nz = df2[df2.instructions_relative_base_frac > 0]
print(len(nz) / len(df2), gmean(nz.instructions_relative_base_frac))

In [ ]:
cond(df, c='Recurse', m='DistinguishMut', p='Precise').groupby('reached_library').num_instructions.describe()

In [ ]:
140427 / (140427 + 276530)

In [ ]:
cond(df, c='SigOnly', m='DistinguishMut', p='Precise').groupby(['reached_library', 'ins_rel_nonzero']) \
    .instructions_relative_frac.describe()[['count', 'mean', '50%']]

In [ ]:
print(732 / 139695)
print(18267.0 / 258263.0)

In [ ]:
# df2 = cond(df, c='SigOnly', m='DistinguishMut', p='Precise')
# df3 = df2.groupby(['crate', 'function_path']) \
#     .apply(lambda df: df.iloc[0]).reset_index(drop=True)
num_slices = df2.groupby(['crate']).size().rename('num_slices')
avg_ins_per_func = df3.groupby('crate').num_instructions.mean() \
    .rename('avg_ins_per_func')
num_funcs = df2.groupby(['crate']).size().rename('num_funcs')

q = df2.groupby(['crate', 'function_path']).size()
avg_slices_per_func = q[q > 0] \
    .groupby('crate').mean().rename('avg_slices_per_func')
pd.concat(
    [num_slices, avg_ins_per_func, num_funcs, avg_slices_per_func], 
    axis=1).sort_values('num_slices')